# CS 447 Homework 1 $-$ Language Models & Morphological Transduction
In this homework we will study some traditional appraoches to a few natural language tasks. First, you will build some n-gram language models on a corpus of Wikipedia articles, and then you will design a finite-state transducer for verb conjugation in Spanish.

This notebook is designed to be run in Google Colab. Navigate to <TT>colab.research.google.com</TT> and upload this notebook. Then follow the instructions in the notebook to do the assignent.

To run the notebook, you will need to connect to a Runtime. For this homework, all you need is a CPU. You can change the runtime by going to <TT>Runtime > Change runtime type</TT> and selecting <TT>None</TT> in the <TT>Hardware Accelerator</TT> field. We encourage you to disconnect from the runtime when you are not using it, as Google Colab can limit your resources if you overuse them.

You can read more about Google Colab at https://research.google.com/colaboratory/faq.html.

We have imported all the libraries you need to do this homework. <b>You should not import any extra libraries.</b> If you do, the autograder will fail to run your code.

#Part 1: Language Models [60 points]

Here, you will train some <b>n-gram language models</b> on WikiText-2, a corpus of high-quality Wikipedia articles. The dataset was originally introduced in the following paper: https://arxiv.org/pdf/1609.07843v1.pdf. A raw version of the data can easily be viewed here: https://github.com/pytorch/examples/tree/master/word_language_model/data/wikitext-2.

Unfortunately, you have to install the <TT>torchdata</TT> package on the Colab machine in order to access the data. To do this, run the cell below (you may need to click the "Restart Runtime" button when it finishes). You will have to do this every time you return to work on the homework.

In [ ]:
!pip install torchdata==0.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


## Preprocessing the Data

To make your models more robust, it is necessary to perform some basic preprocessing on the corpora. <i>You do not need to edit this code.</i>

* <b>Sentence splitting:</b>&nbsp;&nbsp;&nbsp;&nbsp;In this homework, we are interested in modeling individual sentences, rather than longer chunks of text such as paragraphs or documents. The WikiTest dataset provides paragraphs; thus, we provide a simple method to identify individual sentences by splitting paragraphs at punctuation tokens (".",  "!",  "?").

* <b>Sentence markers:</b>&nbsp;&nbsp;&nbsp;&nbsp;For both training and testing corpora, each sentence must be surrounded by a start-of-sentence (`<s>`) and end-of-sentence marker (`/s`). These markers will allow your models to generate sentences that have realistic beginnings and endings.

* <b>Unknown words:</b>&nbsp;&nbsp;&nbsp;&nbsp;In order to deal with unknown words in the test corpora, all words that do not appear in the vocabulary must be replaced with a special token for unknown words (`<UNK>`) before estimating your models. The WikiText dataset has already done this, and you can read about the method in the paper above. When unknown words are encountered in the test corpus, they should be treated as that special token instead.

We provide you with preprocessing code here, and you should not modify it.

After the preprocessing, you may assume that all words in the test set appear in the training set, as this code has already replaced the unseen tokens with `<UNK>`.

In [ ]:
# Constants (feel free to use these in your code, but do not change them)
START = "<s>"   # Start-of-sentence token
END = "</s>"    # End-of-sentence-token
UNK = "<UNK>"   # Unknown word token

In [ ]:
### DO NOT EDIT ###

import torchtext
import random
import sys

def preprocess(data, vocab=None):
    final_data = []
    lowercase = "abcdefghijklmnopqrstuvwxyz"
    for paragraph in data:
        paragraph = [x if x != '<unk>' else UNK for x in paragraph.split()]
        if vocab is not None:
            paragraph = [x if x in vocab else UNK for x in paragraph]
        if paragraph == [] or paragraph.count('=') >= 2: continue
        sen = []
        prev_punct, prev_quot = False, False
        for word in paragraph:
            if prev_quot:
                if word[0] not in lowercase:
                    final_data.append(sen)
                    sen = []
                    prev_punct, prev_quot = False, False
            if prev_punct:
                if word == '"':
                    prev_punct, prev_quot = False, True
                else:
                    if word[0] not in lowercase:
                        final_data.append(sen)
                        sen = []
                        prev_punct, prev_quot = False, False
            if word in {'.', '?', '!'}: prev_punct = True
            sen += [word]
        if sen[-1] not in {'.', '?', '!', '"'}: continue # Prevent a lot of short sentences
        final_data.append(sen)
    vocab_was_none = vocab is None
    if vocab is None:
        vocab = set()
    for i in range(len(final_data)):
        final_data[i] = [START] + final_data[i] + [END]
        if vocab_was_none:
            for word in final_data[i]:
                vocab.add(word)
    return final_data, vocab

def getDataset():
    dataset = torchtext.datasets.WikiText2(root='.data', split=('train', 'valid'))
    train_dataset, vocab = preprocess(dataset[0])
    test_dataset, _ = preprocess(dataset[1], vocab)

    return train_dataset, test_dataset

train_dataset, test_dataset = getDataset()

Run the next cell to see 10 random sentences of the training data.

In [ ]:
if __name__ == '__main__':
    for x in random.sample(train_dataset, 10):
        print (x)

['<s>', 'One', 'dot', 'indicated', 'a', '"', 'minor', 'stop', '"', '(', 'presumably', 'a', 'simple', 'word', 'break', ')', ',', 'two', 'dots', 'marked', 'or', 'separated', '"', 'special', 'words', '"', ',', 'three', 'dots', 'for', 'a', '"', 'bigger', 'stop', '"', '(', 'such', 'as', 'the', '<UNK>', 'name', 'and', 'title', '"', 'the', 'sovereign', 'Alexander', '"', ',', 'below', ',', 'or', 'the', 'title', 'of', 'the', 'Gospel', 'of', 'Matthew', ',', 'above', ')', ',', 'and', 'six', 'dots', 'were', 'to', 'indicate', 'the', 'end', 'of', 'the', 'sentence', '.', '</s>']
['<s>', 'He', 'played', 'his', 'entire', 'career', 'with', 'a', 'piece', 'of', 'shrapnel', 'lodged', 'in', 'his', 'back', 'as', 'a', 'result', 'of', 'this', 'injury', '.', '</s>']
['<s>', 'Baltimore', 'was', 'granted', 'the', 'loan', 'of', 'one', 'of', 'the', 'ships', 'to', 'aid', 'in', 'his', 'defence', 'of', 'the', 'colony', ',', 'as', 'well', 'as', 'a', 'share', 'of', 'the', 'prize', 'money', '.', '</s>']
['<s>', 'Boca', '

## The LanguageModel Class

You will implement 4 types of language models: a <b>unigram</b> model, a <b>smoothed unigram</b> model, a <b>bigram</b> model, and a <b>smoothed bigram</b> model. Each of the models is worth 15 points and extends the following base class. <b>You do not need to implement anything in this class</b>; you will instead implement each of the following methods in the relevant subclass:

* <b>`__init__(self, trainCorpus)`</b>: Train the language model on `trainCorpus`. This will involve calculating relative frequency estimates according to the type of model you're implementing.

* <b>`generateSentence(self)`</b>: <b>[5 points]</b> Return a sentence that is generated by the language model. It should be a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>, where each <TT>w<sup>(i)</sup></TT> is a word in your vocabulary (including <TT>&lt;UNK&gt;</TT> but exlcuding <TT>&lt;s&gt;</TT> and <TT>&lt;&sol;s&gt;</TT>). You may assume that <TT>&lt;s&gt;</TT> starts each sentence (with probability $1$). The following words <TT>w<sup>(1)</sup></TT>, ... , <TT>w<sup>(n)</sup></TT>, <TT>&lt;&sol;s&gt;</TT> are generated according to your language model's distribution. Note that the number of words <TT>n</TT> is not fixed; instead, you should stop the sentence as soon as you generate the stop token <TT>&lt;&sol;s&gt;</TT>.

* <b>`getSentenceLogProbability(self, sentence)`</b>: <b>[5 points]</b> Return the <em> logarithm of the probability</em> of <TT>sentence</TT>, which is again a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>. You should use the natural logarithm $-$ that is, the base-<em>e</em> logarithm. See the note below about performing your calculations in log space.

* <b>`getCorpusPerplexity(self, testCorpus)`</b>: <b>[5 points]</b> You need to compute the perplexity (normalized inverse log probability) of `testCorpus` according to your model. For a corpus $W$ with $N$ words and a bigram model, Jurafsky and Martin tells you to compute perplexity as follows:

$$Perplexity(W) = \Big [ \prod_{i=1}^N \frac{1}{P(w^{(i)}|w^{(i-1)})} \Big ]^{1/N}$$

<b>Implementation Hint:</b> In order to avoid underflow, you will likely need to do all of your calculations in log-space. That is, instead of multiplying probabilities, you should add the logarithms of the probabilities and exponentiate the result:

$$\prod_{i=1}^N P(w^{(i)}|w^{(i-1)}) = \exp\Big (\sum_{i=1}^N \log P(w^{(i)}|w^{(i-1)}) \Big ) $$

Using this property should help you in your implementation of `generateSentence(self)` and `getCorpusPerplexity(self, testCorpus)`.

Feel free to implement helper methods as you wish (either in the base class or in the subclases). <b>But be sure not to change the function signatures of the provided methods</b> (i.e. the function and argument names), or else the autograder will fail.

In [ ]:
import math
import random
from collections import defaultdict

class LanguageModel(object):
    def __init__(self, trainCorpus):
        '''
        Initialize and train the model (i.e. estimate the model's underlying probability
        distribution from the training corpus.)
        '''

        ### DO NOT EDIT ###
        return

    def generateSentence(self):
        '''
        Generate a sentence by drawing words according to the model's probability distribution.
        Note: Think about how to set the length of the sentence in a principled way.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement generateSentence in each subclass.")

    def getSentenceLogProbability(self, sentence):
        '''
        Calculate the log probability of the sentence provided.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getSentenceProbability in each subclass.")

    def getCorpusPerplexity(self, testCorpus):
        '''
        Calculate the perplexity of the corpus provided.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getCorpusPerplexity in each subclass.")

    def printSentences(self, n):
        '''
        Prints n sentences generated by your model.
        '''

        ### DO NOT EDIT ###
        for i in range(n):
            sent = self.generateSentence()
            prob = self.getSentenceLogProbability(sent)
            print('Log Probability:', prob , '\tSentence:',sent)

## <font color='red'>TODO:</font> Unigram Model [15 points]

Here, you will implement each of the 4 functions described above for an <b>unsmoothed unigram</b> model. The probability distribution of a word is given by $\hat P(w)$.

<font color='green'><b>Hints:</b></font>
* <font color='green'>You should use a <b>dictionary</b> to map tokens to their unigram counts.</font>
* <font color='green'>Since you never want to generate the start-of-sentence token `<s>`, you should <b>not</b> include it in your counts.</font>
* <font color='green'>In general, avoid checking for membership in a list (i.e. avoid `x in lst`). Instead, use sets or dictionaries for this purpose $-$ membership checks are much faster on these data structures.</font>
* <font color='green'>Do <b>not</b> modify the training or test corpora by using `.append(...)` or `.pop(...)` on them. This will cause unexpected behavior in the autograder tests, which do not expect you to be changing the data.

In [ ]:
class UnigramModel(LanguageModel):
    def __init__(self, trainCorpus):
        self.wordCount = dict()
        for sentence in trainCorpus:
          for word in sentence:
            if word == '<s>':
              continue
            if word in self.wordCount:
              self.wordCount[word] = self.wordCount[word] + 1
            else:
              self.wordCount[word] = 1

    def generateSentence(self):
        sentence = list()
        sentence.append('<s>')
        track = self.wordCount.copy()
        while True:
            nextWord = random.choices(list(track.keys()), list(track.values()))[0]
            sentence.append(nextWord)
            if nextWord == '</s>':
                break
        return sentence

    def getSentenceLogProbability(self, sentence):
        currProb = 0.0
        track = self.wordCount.copy()
        total = sum(list(track.values()))
        for word in sentence:
            if word == '<s>':
              continue
            freq = track[word]/(total)
            currProb += math.log(freq)
        return currProb

    def getCorpusPerplexity(self, testCorpus):
        #N = sum(list(self.wordCount.values()))
        N = 0
        probSum = 0.0
        for sentence in testCorpus:
            logProb = self.getSentenceLogProbability(sentence)
            probSum += logProb
            N += len(sentence)-1
        return 1/(math.exp(probSum*(1/N)))

We provide you with a testing function that uses very simple training & test corpora (you could compute probability/perplexity by hand if you wanted to). This is just a <b>sanity check</b> $-$ passing this test does not guarantee you a perfect score in the autograder; this is simply to help you debug your model.

In [ ]:
def sanityCheck(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}

    #	Read in the test corpus
    train_corpus = ["By the Late Classic , a network of few <unk> ( few <unk> ) linked various parts of the city , running for several kilometres through its urban core .",
    "Few people realize how difficult it was to create Sonic 's graphics engine , which allowed for the incredible rate of speed the game 's known for ."]
    test_corpus = ["Classic few parts of the game allowed for few <unk> <unk> incredible city .",
                   "Few <unk> realize the difficult network , which linked the game to Sonic ."]
    train_corpus, _ = preprocess(train_corpus)
    test_corpus, _ = preprocess(test_corpus)
    sentence = preprocess(["Sonic was difficult ."])[0][0]

    # These are the correct answers (don't change them!)
    if model_type == "unigram":
       senprobs = [-19.08542845, -114.5001481799, -108.7963657053, -53.6727664115, -55.4645258807]
       trainPerp, testPerp = 41.3308239726, 38.0122981569
       model = UnigramModel(train_corpus)
    elif model_type == "smoothed-unigram":
       senprobs = [-19.0405293515, -115.3479413049, -108.9114348746, -54.8190029616, -55.8122547346]
       trainPerp, testPerp = 41.9994393615, 39.9531928383
       model = SmoothedUnigramModel(train_corpus)
    elif model_type == "bigram":
       senprobs = [-float('inf'), -10.3450917073, -9.2464794186, -float('inf'), -float('inf')]
       trainPerp, testPerp = 1.3861445461, float('inf')
       model = BigramModel(train_corpus)
    elif model_type == "smoothed-bigram":
       senprobs = [-16.355820202, -76.0026113319, -74.2346475108, -47.2885760372, -51.2730261907]
       trainPerp, testPerp = 12.2307627397, 26.7193157699
       model = SmoothedBigramModelAD(train_corpus)
    else: assert False, 'Invalid model_type'

    print("--- TEST: generateSentence() ---")
    modelSen = model.generateSentence()
    senTestPassed = isinstance(modelSen, list) and len(modelSen) > 1 and isinstance(modelSen[0], str)
    if senTestPassed:
        print ("Test generateSentence() passed!")
    else:
        print ("Test generateSentence() failed; did not return a list of strings...")

    print("\n--- TEST: getSentenceLogProbability(...) ---")
    sentences = [sentence, *train_corpus, *test_corpus]
    failed = 0
    for i in range(len(sentences)):
        sen, correct_prob = sentences[i], senprobs[i]
        prob = round(model.getSentenceLogProbability(sen), 10)
        print("Correct log prob.:", correct_prob, '\tYour log prob.:', prob, '\t', 'PASSED' if prob == correct_prob else 'FAILED', '\t', sen)
        if prob != correct_prob: failed+=1

    if not failed:
        print ("Test getSentenceProbability(...) passed!")
    else:
        print("Test getSentenceProbability(...) failed on", failed, "sentence" if failed == 1 else 'sentences...')

    print("\n--- TEST: getCorpusPerplexity(...) ---")
    train_perp = round(model.getCorpusPerplexity(train_corpus), 10)
    test_perp = round(model.getCorpusPerplexity(test_corpus), 10)

    print("Correct train perp.:", trainPerp, '\tYour train perp.:', train_perp, '\t', 'PASSED' if trainPerp == train_perp else 'FAILED')
    print("Correct test perp.:", testPerp, '\tYour test perp.:', test_perp, '\t', 'PASSED' if testPerp == test_perp else 'FAILED')
    train_passed, test_passed = train_perp == trainPerp, test_perp == testPerp
    if train_passed and test_passed:
        print("Test getCorpusPerplexity(...) passed!")
    else:
        print("Test getCorpusPerplexity(...) failed on", "the training corpus and the testing corpus..." if not train_passed and not test_passed else "the testing corpus..." if not test_passed else "the training corpus...")

if __name__=='__main__':
    sanityCheck('unigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -19.08542845 	Your log prob.: -19.08542845 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -114.5001481799 	Your log prob.: -114.5001481799 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -108.7963657053 	Your log prob.: -108.7963657053 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -53.6727664115 	Your log prob.: -53.6727664115 	 PASSED 	 ['<s>', 'Classic', 'few', '

Next, we provide you with another <b>sanity check</b> that trains your model on the *entire* training set, and tests your functions on a small corpus (10 sentences) of *real* test data.

If your code is inefficient, you will likely see that this cell is taking too long. This cell is expected to run in fewer than <b>10 seconds</b>, so if it takes significantly longer than that, you should probably inspect your code for efficiency issues.

In [ ]:
def sanityCheckFullDataset(model_type):
    model = UnigramModel(train_dataset)
    idxes = list(range(75,7500, 800))
    small_test_corpus = [test_dataset[idx] for idx in idxes]
    if model_type == 'unigram':
        senprobs = [-80.7782190984, -174.4769654449, -136.455148267, -225.5890741503, -719.0142129846, -236.350443633, -126.0056604204, -47.3424655612, -47.7775372096, -138.8159941929]
        testPerp = 881.0132848704
        model = UnigramModel(train_dataset)
    elif model_type == 'smoothed-unigram':
        senprobs = [-80.8423009715, -174.5131424172, -136.3181234818, -225.357454098, -719.1543898871, -236.6682968913, -126.1965419509, -47.4369338195, -47.7692144935, -138.542462715]
        testPerp = 881.6105352831
        model = SmoothedUnigramModel(train_dataset)
    elif model_type == 'bigram':
        senprobs = [-float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -32.1502020637, -float('inf'), -float('inf')]
        testPerp = float ('inf')
        model = BigramModel(train_dataset)
    elif model_type == 'smoothed-bigram':
        senprobs = [-61.3754065648, -141.9754903887, -107.0849366076, -168.4944718788, -619.9409055374, -195.8159911677, -86.3762008156, -32.4764801981, -48.124714509, -124.687107856]
        testPerp = 261.4247123506
        model = SmoothedBigramModelAD(train_dataset)
    else: assert False, 'Invalid model_type'
    print("\n--- TEST: getSentenceLogProbability(...) ---")
    failed = 0
    for i in range(len(small_test_corpus)):
        sen, correct_prob = small_test_corpus[i], senprobs[i]
        prob = round(model.getSentenceLogProbability(sen), 10)
        print("Correct log prob.:", correct_prob, '\tYour log prob.:', prob, '\t', 'PASSED' if prob == correct_prob else 'FAILED', '\t', sen)
        if prob != correct_prob: failed+=1

    if not failed:
        print ("Test getSentenceProbability(...) passed!")
    else:
        print("Test getSentenceProbability(...) failed on", failed, "sentence" if failed == 1 else 'sentences...')

    print("\n--- TEST: getCorpusPerplexity(...) ---")
    test_perp = round(model.getCorpusPerplexity(small_test_corpus), 10)

    print("Correct test perp.:", testPerp, '\tYour test perp.:', test_perp, '\t', 'PASSED' if testPerp == test_perp else 'FAILED')
    test_passed = test_perp == testPerp
    if test_passed:
        print("Test getCorpusPerplexity(...) passed!")
    else:
        print("Test getCorpusPerplexity(...) failed on the testing corpus...")

if __name__=='__main__':
    sanityCheckFullDataset('unigram')


--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -80.7782190984 	Your log prob.: -80.7782190984 	 PASSED 	 ['<s>', 'He', 'was', '<UNK>', 'at', '<UNK>', 'College', ',', 'Hobart', ',', 'and', '<UNK>', 'in', '1932', '.', '</s>']
Correct log prob.: -174.4769654449 	Your log prob.: -174.4769654449 	 PASSED 	 ['<s>', 'Despite', 'being', 'a', 'rare', 'Grade', '9', 'player', 'on', 'the', 'senior', 'team', ',', 'he', 'was', 'one', 'of', 'the', 'Knights', "'", 'two', 'leading', 'rushers', 'that', 'year', '.', '</s>']
Correct log prob.: -136.455148267 	Your log prob.: -136.455148267 	 PASSED 	 ['<s>', 'Burke', "'s", 'total', 'was', 'a', 'school', 'record', 'for', 'the', 'Big', 'Ten', 'Conference', 'Men', "'s", 'Basketball', 'Tournament', '.', '</s>']
Correct log prob.: -225.5890741503 	Your log prob.: -225.5890741503 	 PASSED 	 ['<s>', 'The', 'route', 'turns', 'to', 'the', 'northeast', ',', 'passing', 'near', 'the', '<UNK>', 'Leaf', 'Lakes', 'residential', 'development', ',', 'bef

Finally, you can train your model on the full WikiText corpus, and evaluate it on the held-out test set.

In [ ]:
def runModel(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}
    # Read the corpora
    if model_type == 'unigram':
        model = UnigramModel(train_dataset)
    elif model_type == 'bigram':
        model = BigramModel(train_dataset)
    elif model_type == 'smoothed-unigram':
        model = SmoothedUnigramModel(train_dataset)
    else:
        model = SmoothedBigramModelAD(train_dataset)

    print("--------- 5 sentences from your model ---------")
    model.printSentences(5)

    print ("\n--------- Corpus Perplexities ---------")
    print ("Training Set:", model.getCorpusPerplexity(train_dataset))
    print ("Testing Set:", model.getCorpusPerplexity(test_dataset))

if __name__=='__main__':
    runModel('unigram')

--------- 5 sentences from your model ---------
Log Probability: -341.56440491235276 	Sentence: ['<s>', 'had', 'MTV', 'escorting', 'tailed', 'in', 'the', ',', '.', 'over', 'individuals', 'in', 'wrought', 'backup', '(', 'reggaeton', 'on', 'in', 'Oscar', 'to', 'the', 'them', '.', 'was', 'after', 'a', 'and', 'No.', 'considered', '<UNK>', 'more', 'Humphrey', 'is', 'the', 'that', ',', 'Company', 'America', ',', 'listed', 'muscles', 'actin', 'Song', 'is', '100', 'State', '<UNK>', 'to', 'of', 'The', 'when', 'a', '</s>']
Log Probability: -416.82711136402764 	Sentence: ['<s>', 'Shiva', 'notes', 'over', 'issue', ',', 'to', 'in', 'favor', 'mile', 'was', 'Moon', 'It', 'Saint', '.', ':', ',', 'AP', '.', 'by', ',', 'a', 'indicating', ',', 'country', '.', 'biography', ',', '&', 'unnecessary', 'paintings', 'in', ';', 'Bowl', 'a', ')', 'first', 'passed', '<UNK>', 'that', 'and', 'rails', 'to', 'to', 'inadvertently', 'Christopher', 'had', 'H.', 'the', 'down', 'clock', 'Duke', 'flushed', 'the', 'full', 'b

## <font color='red'>TODO:</font> Smoothed Unigram Model [15 points]

Here, you will implement each of the 4 functions described above for a <b>unigram</b> model with <b>Laplace (add-one) smoothing</b>. The probability distribution of a word is given by $P_L(w)$. This type of smoothing takes away some of the probability mass for observed events and assigns it to unseen events.

In order to smooth your model, you will need the number of words in the corpus, $N$, and the number of word types, $S$. The distinction between these is meaningful: $N$ indicates the number of word instances, where $S$ refers to the size of our vocabulary. For example, the sentence <em>the cat saw the dog</em> has four word types (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>dog</em>), but five word tokens (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>the</em>, <em>dog</em>). The token <em>the</em> appears twice in the sentence, but they share the same type <em>the</em>.

If $c(w)$ is the frequency of $w$ in the training data, you can compute $P_L(w)$ as follows:

$$P_L(w)=\frac{c(w)+1}{N+S}$$

<font color='green'><b>Hints:</b></font>
* <font color='green'>You may find it convenient to make your `SmoothedUnigramModel` inherit your `UnigramModel`, and then override the function(s) that need to be changed.</font>

In [ ]:
class SmoothedUnigramModel(LanguageModel):
    def __init__(self, trainCorpus):
        self.wordCount = dict()
        for sentence in trainCorpus:
          for word in sentence:
            if word == '<s>':
              continue
            if word in self.wordCount:
              self.wordCount[word] = self.wordCount[word] + 1
            else:
              self.wordCount[word] = 1
        numWords = sum(list(self.wordCount.values()))
        vocabSize = len(list(self.wordCount.keys()))
        for word in self.wordCount:
            occ = self.wordCount[word]
            self.wordCount[word] = (occ+1.0)#/(numWords+vocabSize)

    def generateSentence(self):

        sentence = list()
        sentence.append('<s>')
        track = self.wordCount.copy()
        while True:
            nextWord = random.choices(list(track.keys()), list(track.values()))[0]
            sentence.append(nextWord)
            if nextWord == '</s>':
                break
        return sentence

    def getSentenceLogProbability(self, sentence):

        currProb = 0.0
        track = self.wordCount.copy()
        total = sum(list(track.values()))
        for word in sentence:
            if word == '<s>':
              continue
            freq = track[word]/(total)
            currProb += math.log(freq)
        return currProb

    def getCorpusPerplexity(self, testCorpus):

        N = 0
        probSum = 0.0
        for sentence in testCorpus:
            logProb = self.getSentenceLogProbability(sentence)
            probSum += logProb
            N += len(sentence)-1
        return 1/(math.exp(probSum*(1/N)))

In [ ]:
if __name__=='__main__':
    sanityCheck('smoothed-unigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -19.0405293515 	Your log prob.: -19.0405293515 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -115.3479413049 	Your log prob.: -115.3479413049 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -108.9114348746 	Your log prob.: -108.9114348746 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -54.8190029616 	Your log prob.: -54.8190029616 	 PASSED 	 ['<s>', 'Classic', 'few

Since the next sanity check trains your model on the *entire* training set, you will likely see that it is taking too long if you have inefficiences in your code. This cell is expected to run in fewer than <b>10 seconds</b>, so if it takes significantly longer than that, you should probably inspect your code for efficiency issues.

In [ ]:
if __name__=='__main__':
    sanityCheckFullDataset('smoothed-unigram')


--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -80.8423009715 	Your log prob.: -80.8423009715 	 PASSED 	 ['<s>', 'He', 'was', '<UNK>', 'at', '<UNK>', 'College', ',', 'Hobart', ',', 'and', '<UNK>', 'in', '1932', '.', '</s>']
Correct log prob.: -174.5131424172 	Your log prob.: -174.5131424172 	 PASSED 	 ['<s>', 'Despite', 'being', 'a', 'rare', 'Grade', '9', 'player', 'on', 'the', 'senior', 'team', ',', 'he', 'was', 'one', 'of', 'the', 'Knights', "'", 'two', 'leading', 'rushers', 'that', 'year', '.', '</s>']
Correct log prob.: -136.3181234818 	Your log prob.: -136.3181234818 	 PASSED 	 ['<s>', 'Burke', "'s", 'total', 'was', 'a', 'school', 'record', 'for', 'the', 'Big', 'Ten', 'Conference', 'Men', "'s", 'Basketball', 'Tournament', '.', '</s>']
Correct log prob.: -225.357454098 	Your log prob.: -225.357454098 	 PASSED 	 ['<s>', 'The', 'route', 'turns', 'to', 'the', 'northeast', ',', 'passing', 'near', 'the', '<UNK>', 'Leaf', 'Lakes', 'residential', 'development', ',', 'bef

In [ ]:
if __name__=='__main__':
    runModel('smoothed-unigram')

--------- 5 sentences from your model ---------
Log Probability: -80.4961059200074 	Sentence: ['<s>', 'accused', 'three', 'PRG', 'tradition', 'of', 'located', '2008', 'was', 'the', ',', 'two', '</s>']
Log Probability: -228.29138580323166 	Sentence: ['<s>', 'a', 'area', 'in', 'Olympic', 'and', 'Braun', 'RAAF', 'protests', '<UNK>', 'Gaelic', 'the', 'final', 'carried', 'young', 'Anand', '2010', 'enough', '"', '<UNK>', 'do', 'division', 'the', 'Dylan', '.', '.', 'ibotenic', 'reserves', 'by', 'are', 'constant', '</s>']
Log Probability: -506.7019251710034 	Sentence: ['<s>', 'meetings', 'yet', 'tracked', '.', 'sector', 'body', 'and', 'Allen', 'of', 'laid', '.', '.', '@-@', '.', 'to', '<UNK>', 'her', 'German', 'were', 'the', 'judo', 'the', 'and', 'War', '’', 'box', 'to', 'upon', 'Dutchman', 'detectors', 'State', 'river', 'of', ',', '.', 'Theodor', 'retreating', 'metal', 'referendum', 'Isabella', 'greater', 'her', 'controversial', ',', 'number', 'commercial', '–', 'Basement', 'experiences', 'Li

## <font color='red'>TODO:</font> Bigram Model [15 points]

Here, you will implement each of the 4 functions described above for an <b>unsmoothed bigram</b> model. The probability distribution of a word is given by $\hat P(w'|w)$. Thus, the probability of $w_i$ is conditioned on $w_{i-1}$.

<font color='green'><b>Hints:</b></font>
* <font color='green'>You should use a dictionary of dictionaries to store your bigram counts. That is, the outer dictionary should map $w$ to another dictionary that maps $w'$ to the number of times $w'$ occurs after $w$.</font>
* <font color='green'>Do <b>not</b> attempt to iterate over all possible bigrams in your voabulary: <em>only store bigrams that actually occur in your training data.</em> You will run into timeout or out-of-memory issues if you attempt to enumerate all bigrams.</font>
* <font color='green'>Similarly, avoid nested loops over the training data.</font>

In [ ]:
class BigramModel(LanguageModel):
    def __init__(self, trainCorpus):

        self.wordCount = dict()
        for sentence in trainCorpus:
            for i in range(len(sentence)-1):
                firstWord = sentence[i]
                secondWord = sentence[i+1]
                if firstWord in self.wordCount:
                    if secondWord in self.wordCount[firstWord]:
                        self.wordCount[firstWord][secondWord] = self.wordCount[firstWord][secondWord] + 1
                    else:
                        self.wordCount[firstWord][secondWord] = dict()
                        self.wordCount[firstWord][secondWord] = 1
                else:
                    self.wordCount[firstWord] = dict()
                    self.wordCount[firstWord][secondWord] = 1


    def generateSentence(self):

        sentence = list()
        sentence.append('<s>')
        track = self.wordCount.copy()
        lastWord = sentence[0]
        while True:
            nextWord = random.choices(list(track[lastWord].keys()), list(track[lastWord].values()))[0]
            sentence.append(nextWord)
            if nextWord == '</s>':
                break
            lastWord = nextWord
        return sentence

    def getSentenceLogProbability(self, sentence):
        #print(self.wordCount)
        currProb = 0.0
        track = self.wordCount.copy()
        for i in range(len(sentence)-1):
            firstWord = sentence[i]
            secondWord = sentence[i+1]
            total = sum(track[firstWord].values())
            if secondWord in track[firstWord]:
                freq = track[firstWord][secondWord]/(total)
                currProb += math.log(freq)
            else:
                return float('-inf')
        return currProb

    def getCorpusPerplexity(self, testCorpus):

        N = 0
        probSum = 0.0
        for sentence in testCorpus:
            logProb = self.getSentenceLogProbability(sentence)
            probSum += logProb
            N += len(sentence)-1
        temp = math.exp(probSum*(1/N))
        if temp == 0:
            return float('inf')
        return 1/(math.exp(probSum*(1/N)))

In [ ]:
if __name__=='__main__':
    sanityCheck('bigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -10.3450917073 	Your log prob.: -10.3450917073 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -9.2464794186 	Your log prob.: -9.2464794186 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'Classic', 'few', 'parts', 'of', 'the', 'game', 'allowed', 'f

Since the next sanity check trains your model on the *entire* training set, you will likely see that it is taking too long if you have inefficiences in your code. This cell is expected to run in fewer than <b>10 seconds</b>, so if it takes significantly longer than that, you should probably inspect your code for efficiency issues.

In [ ]:
if __name__=='__main__':
    sanityCheckFullDataset('bigram')


--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'He', 'was', '<UNK>', 'at', '<UNK>', 'College', ',', 'Hobart', ',', 'and', '<UNK>', 'in', '1932', '.', '</s>']
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'Despite', 'being', 'a', 'rare', 'Grade', '9', 'player', 'on', 'the', 'senior', 'team', ',', 'he', 'was', 'one', 'of', 'the', 'Knights', "'", 'two', 'leading', 'rushers', 'that', 'year', '.', '</s>']
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'Burke', "'s", 'total', 'was', 'a', 'school', 'record', 'for', 'the', 'Big', 'Ten', 'Conference', 'Men', "'s", 'Basketball', 'Tournament', '.', '</s>']
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'The', 'route', 'turns', 'to', 'the', 'northeast', ',', 'passing', 'near', 'the', '<UNK>', 'Leaf', 'Lakes', 'residential', 'development', ',', 'before', 'coming', 'to', 'an', 'interchange', 'with', 'US', '322', '(', 'Black', 'Horse

In [ ]:
if __name__=='__main__':
    runModel('bigram')

--------- 5 sentences from your model ---------
Log Probability: -19.100225720300188 	Sentence: ['<s>', 'Along', 'with', 'attendees', '.', '</s>']
Log Probability: -11.998261208248897 	Sentence: ['<s>', 'In', 'the', 'series', '.', '</s>']
Log Probability: -63.663075511040034 	Sentence: ['<s>', 'However', ',', 'and', 'Rich', 'Girl', '(', 'known', 'for', 'its', 'admission', ',', 'including', 'grass', 'athletics', '.', '</s>']
Log Probability: -34.20101868197841 	Sentence: ['<s>', 'As', 'the', 'Omaha', 'was', 'truncated', 'seven', 'days', '.', '</s>']
Log Probability: -63.934353432252585 	Sentence: ['<s>', 'The', 'Gazette', ',', 'but', 'as', 'many', 'admirers', ',', 'which', 'she', "'s", 'regime', 'following', 'season', '.', '</s>']

--------- Corpus Perplexities ---------
Training Set: 76.92394608735728
Testing Set: inf


## <font color='red'>TODO:</font> Smoothed Bigram Model [15 points]

Here, you will implement each of the 4 functions described above for a <b>bigram</b> model with <b>absolute discounting</b>. The probability distribution of a word is given by $P_{AD}(w’|w)$.

In order to smooth your model, you need to compute a discounting factor $D$. If $n_k$ is the number of bigrams $w_1w_2$ that appear exactly $k$ times, you can compute $D$ as:

$$D=\frac{n_1}{n_1+2n_2}$$

For each word $w$, you then need to compute the number of bigram types $ww’$ as follows:

$$S(w)=|\{w’\mid c(ww’)>0\}|$$

where $c(ww’)$ is the frequency of $ww’$ in the training data. In other words, $S(w)$ is the number of unique words that follow $w$ at least once in the training data.

Finally, you can compute $P_{AD}(w’|w)$ as follows:

$$P_{AD}(w’|w)=\frac{\max \big (c(ww’)-D,0\big )}{c(w)}+\bigg (\frac{D}{c(w)}\cdot S(w) \cdot P_L(w’)\bigg )$$

where $c(w)$ is the frequency of $w$ in the training data and $P_L(w’)$ is the Laplace-smoothed unigram probability of $w’$.

In [ ]:
class SmoothedBigramModelAD(LanguageModel):
    def getProb(self, word1, word2, wordCountTemp, D, numWords, endingCount):
        cw = sum(wordCountTemp[word1].values())
        sw = len(wordCountTemp[word1].keys())
        if word2 != '</s>':
            plw = (sum(wordCountTemp[word2].values())+1)/(numWords+len(wordCountTemp.keys()))
        else:
            plw = (endingCount+1)/(numWords+len(wordCountTemp.keys()))
        if word2 in wordCountTemp[word1].keys():
            cww = wordCountTemp[word1][word2]
        else:
            cww = 0
        prob = (max(cww-D,0)/(cw)) + (D*sw*plw/cw)
        return prob

    def __init__(self, trainCorpus):

        self.wordCountTemp = dict()
        self.numWords = 0
        self.numSentences = len(trainCorpus)
        for sentence in trainCorpus:
            for i in range(len(sentence)-1):
                self.numWords += 1
                firstWord = sentence[i]
                secondWord = sentence[i+1]
                if firstWord in self.wordCountTemp:
                    if secondWord in self.wordCountTemp[firstWord]:
                        self.wordCountTemp[firstWord][secondWord] = self.wordCountTemp[firstWord][secondWord] + 1
                    else:
                        self.wordCountTemp[firstWord][secondWord] = dict()
                        self.wordCountTemp[firstWord][secondWord] = 1
                else:
                    self.wordCountTemp[firstWord] = dict()
                    self.wordCountTemp[firstWord][secondWord] = 1

        n1 = 0
        n2 = 0
        for word1 in self.wordCountTemp.keys():
            for word2 in self.wordCountTemp[word1].keys():
                occ = self.wordCountTemp[word1][word2]
                if occ == 1:
                    n1 += 1
                elif occ == 2:
                    n2 += 1
        self.D = (n1)/(n1+2*n2)
        self.wordCount = dict()
        for word1 in self.wordCountTemp.keys():
            self.wordCount[word1] = dict()
            for word2 in self.wordCountTemp[word1].keys():
                self.wordCount[word1][word2] = self.getProb(word1,word2,self.wordCountTemp,self.D,self.numWords,self.numSentences)

    def generateSentence(self):

        sentence = list()
        sentence.append('<s>')
        track = self.wordCount.copy()
        lastWord = sentence[0]
        while True:
            nextWord = random.choices(list(track[lastWord].keys()), list(track[lastWord].values()))[0]
            sentence.append(nextWord)
            if nextWord == '</s>':
                break
            lastWord = nextWord
        return sentence

    def getSentenceLogProbability(self, sentence):

        currProb = 0.0
        track = self.wordCount.copy()
        for i in range(len(sentence)-1):
            word1 = sentence[i]
            word2 = sentence[i+1]
            if word2 in track[word1]:
                freq = track[word1][word2]
            else:
                freq = self.getProb(word1,word2,self.wordCountTemp,self.D,self.numWords,self.numSentences)
            currProb += math.log(freq)
        return currProb

    def getCorpusPerplexity(self, testCorpus):

        N = 0
        probSum = 0.0
        for sentence in testCorpus:
            logProb = self.getSentenceLogProbability(sentence)
            probSum += logProb
            N += len(sentence)-1
        temp = math.exp(probSum*(1/N))
        if temp == 0:
            return float('inf')
        return 1/(math.exp(probSum*(1/N)))

In [ ]:
if __name__=='__main__':
    sanityCheck('smoothed-bigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -16.355820202 	Your log prob.: -16.355820202 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -76.0026113319 	Your log prob.: -76.0026113319 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -74.2346475108 	Your log prob.: -74.2346475108 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -47.2885760372 	Your log prob.: -47.2885760372 	 PASSED 	 ['<s>', 'Classic', 'few', 'pa

Since the next sanity check trains your model on the *entire* training set, you will likely see that it is taking too long if you have inefficiences in your code. This cell is expected to run in fewer than <b>10 seconds</b>, so if it takes significantly longer than that, you should probably inspect your code for efficiency issues.

In [ ]:
if __name__=='__main__':
    sanityCheckFullDataset('smoothed-bigram')


--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -61.3754065648 	Your log prob.: -61.3754065648 	 PASSED 	 ['<s>', 'He', 'was', '<UNK>', 'at', '<UNK>', 'College', ',', 'Hobart', ',', 'and', '<UNK>', 'in', '1932', '.', '</s>']
Correct log prob.: -141.9754903887 	Your log prob.: -141.9754903887 	 PASSED 	 ['<s>', 'Despite', 'being', 'a', 'rare', 'Grade', '9', 'player', 'on', 'the', 'senior', 'team', ',', 'he', 'was', 'one', 'of', 'the', 'Knights', "'", 'two', 'leading', 'rushers', 'that', 'year', '.', '</s>']
Correct log prob.: -107.0849366076 	Your log prob.: -107.0849366076 	 PASSED 	 ['<s>', 'Burke', "'s", 'total', 'was', 'a', 'school', 'record', 'for', 'the', 'Big', 'Ten', 'Conference', 'Men', "'s", 'Basketball', 'Tournament', '.', '</s>']
Correct log prob.: -168.4944718788 	Your log prob.: -168.4944718788 	 PASSED 	 ['<s>', 'The', 'route', 'turns', 'to', 'the', 'northeast', ',', 'passing', 'near', 'the', '<UNK>', 'Leaf', 'Lakes', 'residential', 'development', ',', 'b

In [ ]:
if __name__=='__main__':
    runModel('smoothed-bigram')

--------- 5 sentences from your model ---------
Log Probability: -226.67018582173932 	Sentence: ['<s>', 'On', '5', 'million', 'five', 'satellite', ',', 'forcing', 'advertisers', 'to', 'have', 'an', 'RAF', 'on', 'a', 'complex', ',', 'in', 'the', 'decency', 'due', 'to', 'the', 'film', ',', 'a', 'goal', 'to', '<UNK>', 'in', 'an', 'asteroid', ',', 'displacement', 'of', 'the', 'fox', '<UNK>', 'have', 'the', 'family', 'and', 'Karaoke', 'Revolution', 'placed', 'in', 'a', '<UNK>', 'Central', 'Park', '.', '</s>']
Log Probability: -49.55214836848939 	Sentence: ['<s>', 'Mitchell', ',', 'in', 'Australia', 'as', 'the', 'song', 'to', 'gain', 'control', '.', '</s>']
Log Probability: -154.8467919688782 	Sentence: ['<s>', 'Most', 'of', 'where', 'as', 'it', 'proper', 'construction', 'project', 'profound', 'feeling', 'on', '15', ':', 'A', 'new', 'restaurant', '"', 'Cambodia', 'and', 'comedy', 'and', '<UNK>', 'study', 'in', 'the', 'DAG', 'in', 'four', 'children', '.', '</s>']
Log Probability: -47.85705473

## Food for Thought
We provide you some questions to think about. <b>You do not need to answer these questions, but we encourage you to give them some thought.</b>
<ol>
<li>When generating sentences with the unigram model, what controls the length of the generated sentences? How does this differ from the sentences produced by the bigram models?
<li>Consider the probability of the generated sentences according to your models. Do your models assign drastically different probabilities to the different sets of sentences? Why do you think that is?
<li>Look back at the sentences generated using your models. In your opinion, which model produces better / more realistic sentences?
<li>For each of the four models, which test corpus has the highest perplexity? Why?
<li> Why do you think it might be a bad idea to use Laplace (add-one) smoothing for a bigram model? How does the absolute discounting method help?
</ol>

# Part 2: Finite State Transducers [40 points]

Here, you will implement a <b>finite state transducer</b> (FST), which transduces the infinitive form of Spanish verbs to the preterite (past tense) form in the 3rd person singular. You will be graded according to how well your transducer performs on a hidden test dataset.

Run the cell below to pull the data you can use to develop your FST.

## Provided Code
We provide you with a class <TT>FST</TT>, which is a module for constructing and evaluating FST's. You shouldn't modify any of this code. Here is a description of the most useful methods of this module:


* <b>`FST(self, initialStateName)`</b>: Instantiate an FST with an initial (non-accepting) state named `initialStateName`

* <b>`addState(self, name, isFinal=False)`</b>: Add a state `name` to the FST; by default, `isFinal=False` and so the state is not an accepting state.

* <b>`addTransition(self, inStateName, inString, outString, outStateName)`</b>: Add a transition between state `inStateName` and state `outStateName`, where both of these states already exist in the FST. The FST can traverse this transition after reading `inString`, and outputs `outString` when it does so.

* <b>`addSetTransition(self, inStateName, inStringSet, outStateName)`</b>: Add a transition between state `inStateName` and state `outStateName` for each character in `inStringSet`. For each transition, the FST outputs the same character it reads.

The cell also contains a method to read the data and a scoring function that you will call in the "Test your FST" section.

<b>Do not edit any of the code in this cell!</b>


In [ ]:
### DO NOT EDIT ###

import os
import pandas as pd
from tabulate import tabulate
pdtabulate=lambda df:tabulate(df,headers='keys',tablefmt='psql', showindex=False)

def readVerbFile(file):
    url='https://drive.google.com/u/0/uc?id=1U6vrmAKep0hDscPrCZ7yJKjFQmPQsfG2&export=download'
    df = pd.read_csv(url, header=None)

    return df.values.tolist()

verbs = readVerbFile('verbsList.csv')

def testFST(print_examples = 'all'):
    assert print_examples in {'all', 'incorrect', 'none'}, "print_examples must be 'all', 'incorrect', or 'none'"
    rule_classes = [('0a*', "0a* Regular -Car verb (stem end in consonant)", 'hablar      ==>  habló'), ('0b*', "0b* Regular -Var verb (stem end in vowel)", 'pasear      ==>  paseó'), ('0c*', "0c* Regular -er verb", 'comer       ==>  comió'), ('0d*', "0d* Regular -ir verb (excluding -guir, -quir)", 'abrir       ==>  abrió'), ('1a', "1a  Verbs in -ñer", 'tañer       ==>  tañó'), ('1b', "1b  Verbs in -ñir (excluding -eñir)", 'gañir       ==>  gañó'), ('2a', "2a  Verbs in -Ver", 'leer        ==>  leyó'), ('2b', "2b  Verbs in -Vir", 'construir   ==>  construyó'), ('2c*', "2c* Verbs in -guir (excluding -eguir)", 'distinguir  ==>  distinguió'), ('2d*', "2d* Verbs in -quir", 'delinquir   ==>  delinquió'), ('3a', "3a  Verbs in -eCir", 'pedir       ==>  pidió'), ('3b', "3b  Verbs in -eCCir", 'sentir      ==>  sintió'), ('3c', "3c  Verbs in -eCCCir", 'henchir     ==>  hinchió'), ('3d', "3d  Verbs in -eguir", 'seguir      ==>  siguió'), ('3e', "3e  Verbs in -eñir", 'heñir       ==>  hiñó')]

    f = buildFST()
    myParses = f.parseInputList([x[0] for x in verbs])
    scores, totals, examples = {}, {}, []

    for i in range(len(verbs)):
        lemma, form, clas = verbs[i]
        output = myParses[i]
        scores[clas] = scores.get(clas, 0)
        totals[clas] = totals.get(clas, 0) + 1
        if print_examples == 'all' or print_examples == 'incorrect' and form != output:
            examples += [(lemma, form, output, 'CORRECT' if form == output else 'INCORRECT')]
        if form == output: scores[clas] += 1

    if print_examples != 'none' and len(examples) > 0:
        examples = pd.DataFrame.from_records(examples, columns = ['Input', 'Correct Output', 'Returned Output', 'Result'])
        print(pdtabulate(examples))

    data= []

    # We use a scoring method that accounts for (1) the fact that the default FST gets many verbs correct
    #                                           (2) the class inbalance among the different rules
    # p_scores is for verbs that the default FST gets correct, q_scores for verbs it gets wrong
    # Each contains a list of accuracies for each category group (for example, all of the examples where rule 2 applies are in one group)
    p_scores, q_scores = {'0*': [], '2*': []}, {'1': [], '2': [], '3': []}
    for clas, msg, ex in rule_classes:
        acc = scores[clas]/totals[clas]
        data += [(msg, ex, scores[clas], totals[clas], 100*acc)]
        if '0' in clas: p_scores['0*'].append(acc)
        elif '1' in clas: q_scores['1'].append(acc)
        elif '2' in clas and '*' not in clas: q_scores['2'].append(acc)
        elif '2' in clas and '*' in clas: p_scores['2*'].append(acc)
        elif '3' in clas:
            if clas in {'3a', '3b', '3c'}: q_scores['3'] += [acc, acc, acc] # Weight these higher than -eguir and -eñir
            else: q_scores['3'].append(acc)
        else: assert False, 'should not get here ' + clas

    p_scores = [sum(v) / len(v) for k, v in p_scores.items()] # Get the average for each category group
    q_scores = [sum(v) / len(v) for k, v in q_scores.items()]

    p_scores = [p_scores[0]] * 3 + [p_scores[1]] # Weight 0* higher than 2*

    # Score by averaging the p and q scores, then applying grading formula
    p, q = sum(p_scores) / len(p_scores), sum(q_scores) / len(q_scores)
    final_score = q/2 * (1+p) # Score will be 0 when p is 100% and q is 0%; score will be 100% when p and q are both 100%

    print('\nScorecard:')
    data = pd.DataFrame.from_records(data, columns = ['Category', 'Example', 'Correct', 'Total', 'Accuracy (%)'])
    print(pdtabulate(data))

    print("Overall Score:", str(final_score*100) + '%')

class Transition:
    # string_in
    # string_out
    def __init__(self, inState, inString, outString, outState):
        self.state_in = inState
        self.string_in = inString
        self.string_out = outString
        self.state_out = outState

    def equals(self, t):
        if self.state_in == t.state_in \
        and self.string_in == t.string_in \
        and self.string_out == t.string_out \
        and self.state_out == t.state_out:
            return True
        else:
            return False

class FSTstate:
    # id: an integer ID of the state
    # isFinal: is this a final state?
    def __init__(self, n, isF, fst):
        self.id = n
        self.isFinal = isF
        self.transitions = dict() # map inStrings to a set of all possible transitions
        self.FST = fst

    def addTransition(self, inString, outString, outState):
        newTransition = Transition(self, inString, outString, outState)
        if inString in self.transitions:
            for t in self.transitions[inString]:
                if t.equals(newTransition):
                    return
            self.transitions[inString].add(newTransition)
        else:
            self.transitions[inString] = set([])
            self.transitions[inString].add(newTransition)

    def parseInputFromStartState(self, inString):
        parseTuple = ("", self.id)
        parses = []
        (accept, stringParses) = self.parseInput(inString)
        if accept:
            for p in stringParses:
                completeParse = [parseTuple]
                completeParse.extend(p)
                parses.append(completeParse)
        return (accept, parses)

    def parseInput(self, inString):
        parses = []
        isAccepted = True

        DEBUG = False
        if DEBUG:
            print("parseInput: state: ", self.id, " parsing: " , inString)

        # Case 1: no suffix
        if inString == "":
            epsilonParses = []
            epsilonAccepted = False
            # try all epsilon transitions
            if "" in self.transitions:
                transSet = self.transitions[""]
                for t in transSet:
                    outString = t.string_out
                    toStateID = t.state_out
                    toState = self.FST.allStates[toStateID]
                    parseTuple = (outString, toStateID)
                    (suffixAccepted, suffixParses) = toState.parseInput(inString)
                    if suffixAccepted:
                        epsilonAccepted = True
                        if suffixParses == []: #accepts.
                            parse_s = [parseTuple]
                            epsilonParses.append(parse_s)
                        else:
                            for s in suffixParses:
                                parse_s = [parseTuple]
                                parse_s.extend(s)
                                epsilonParses.append(parse_s)
            # if epsilon is accepted, add all its parses
            if epsilonAccepted:
                parses.extend(epsilonParses)
            # if this is a final state, add an empty parse
            if self.isFinal or parses != []:
                if DEBUG:
                    print("Accepted in state ", self.id)
                return (True, parses)
            else:
                if DEBUG:
                    print("Rejected in state ", self.id)
                return (False, None)
        # case 2: non-empty suffix: there needs to be one suffix that parses!)
        hasAcceptedSuffix = False;
        for i in range(0,len(inString)+1):
            prefix = inString[0:i]
            suffix = inString[i:len(inString)]
            if DEBUG:
                print("\t prefix: \'", prefix, "\' I=", i)
            if prefix in self.transitions:
                if DEBUG:
                     print("\t prefix: ", prefix,  "suffix: ", suffix, "I=", i)
                transSet = self.transitions[prefix]
                for t in transSet:
                    outString = t.string_out
                    toStateID = t.state_out
                    toState = self.FST.allStates[toStateID]
                    parseTuple = (outString, toStateID)
                    (suffixAccepted, suffixParses) = toState.parseInput(suffix)
                    if suffixAccepted:
                        hasAcceptedSuffix = True
                        if suffixParses == []:
                            parse_s = [parseTuple]
                            parses.append(parse_s)
                            thisPrefixParses = True
                        for s in suffixParses:
                            parse_s = [parseTuple]
                            parse_s.extend(s)
                            parses.append(parse_s)
        if hasAcceptedSuffix:
            return (True, parses)
        else:
            return (False, None)



    def printState(self):
        if self.isFinal:
            FINAL = "FINAL"
        else: FINAL = ""
        print("State", self.id, FINAL)
        for inString in self.transitions:
            transList = self.transitions[inString]
            for t in transList:
                print("\t", inString, ":", t.string_out, " => ", t.state_out)




class FST:
    def __init__(self, initialStateName="q0"):
        self.nStates = 0
        self.initState = FSTstate(initialStateName, False, self)
        self.allStates = dict()
        self.allStates[initialStateName] = self.initState

    def addState(self, name, isFinal=False):
        if name in self.allStates:
            print("ERROR addState: state", name, "exists already")
            sys.exit()
        elif len(self.allStates) >= 30:
            print("ERROR addState: you can't have more than 30 states")
            sys.exit()
        else:
            newState = FSTstate(name, isFinal, self)
            self.allStates[name] = newState

    def addTransition(self, inStateName, inString, outString, outStateName):
        if (len(inString) > 1):
            print("ERROR: addTransition: input string ", inString, " is longer than one character")
            sys.exit()
        if inStateName not in self.allStates:
            print("ERROR: addTransition: state ", inStateName, " does not exist")
            sys.exit()
        if outStateName not in self.allStates:
            print("ERROR: addTransition: state ", outStateName, " does not exist")
            sys.exit()
        inState = self.allStates[inStateName]
        inState.addTransition(inString, outString, outStateName)

    # epsilon:epsilon
    def addEpsilonTransition(self, inStateName, outStateName):
        if inStateName not in self.allStates:
            print("ERROR: addEpsilonTransition: state ", inStateName, " does not exist")
            sys.exit()
        if outStateName not in self.allStates:
            print("ERROR: addEpsilonTransition: state ", outStateName, " does not exist")
            sys.exit()
        if inStateName == outStateName:
            print("ERROR: we don't allow epsilon loops")
            sys.exit()
        inState = self.allStates[inStateName]
        inState.addTransition("", "", outStateName)

    # map every element in inStringSet to itself
    def addSetTransition(self, inStateName, inStringSet, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, s, outStateName)

    # map string to itself
    def addSelfTransition(self, inStateName, inString, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", outStateName, " does not exist")
            sys.exit()
         self.addTransition(inStateName, inString, inString, outStateName)

    # map every element in inStringSet to outString
    def addSetToStringTransition(self, inStateName, inStringSet, outString, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetDummyTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetDummyTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, outString, outStateName)


    # map every element in inStirngSet to outString
    def addSetEpsilonTransition(self, inStateName, inStringSet, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetEpsilonTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetEpsionTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, "", outStateName)

    def parseInput(self, inString):
        SHOW_STATES = False#True
        inString = inString.rstrip('\n')
        (canParse, allParses)  = self.initState.parseInputFromStartState(inString)
        allParsesAsString = ""
        if canParse:
            for parse in allParses:
                for tuple in parse:
                    outString, outState = tuple
                    allParsesAsString += outString
                if SHOW_STATES:
                    allParsesAsString += "\t  States: "
                    i = 0
                    for tuple in parse:
                        i += 1
                        outString, outState = tuple
                        allParsesAsString += outState
                        if i < len(parse):
                            allParsesAsString += " => "
                    allParsesAsString += "; "

            return True, allParsesAsString
        else:
            return False, "FAIL"

    def printFST(self):
        print("Printing FST", str(self))
        for stateID in self.allStates:
            state = self.allStates[stateID]
            state.printState()

    def parseInputList(self, verbList):
        #with open(fileName, "r") as f:
        nParses = 0
        totalStrings = 0
        res = []
        for verb in verbList:#f:
            totalStrings += 1
            canParse, parse = self.parseInput(verb)
            res += [parse]
            if canParse:
                nParses += 1
        fraction = nParses/totalStrings
        print(nParses, "/", totalStrings, "=", str(fraction*100)+'%', "of examples parsed")
        return res

## Spanish Past Tense Transduction
Here we provide you with a specification for producing the 3rd person singular of the Spanish preterite tense from the verb infinitive.<br>You may assume that all Spanish infinitives consist of a <em>stem</em> and <em>ending</em>, which is always <TT>-ar</TT>, <TT>-er</TT>, or <TT>-ir</TT>.<br>For example, the stem of the verb <TT>hablar</TT> is <TT>habl-</TT> and the ending is <TT>-ar</TT>.

If the verb ends in <TT>-ar</TT>, then the preterite form consists of stem + <TT>ó</TT>. If the verb ends in <TT>-er</TT> or <TT>-ir</TT>, then the preterite form consists of stem + <TT>ió</TT>:

<ul>
<TT>hablar ==> habló&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;comer ==> comió&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;escribir ==> escribió</TT>
</ul>

But there are a number of exceptions. In particular, you will need to implement the following rules.<br><b>Important:</b> These rules apply <em>only</em> to verbs ending in <TT>-er</TT> or <TT>-ir</TT>.
<ol>
<li><b>Stems ending in <TT>ñ</TT>:</b> If the stem ends in <TT>ñ</TT>, the ending is <TT>-ó</TT> rather than <TT>-ió</TT>:
<ul>
<TT>tañer ==> tañó&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;gruñir ==> gruñó</TT>
</ul>
<em>Note:</em> This rule does <em>not</em> affect stems ending in regular <TT>n</TT>; it only affects stems in <T>ñ</TT> (<em>n</em> with a tilde).<br>
<li><b>Stems ending in a vowel:</b> If the stem ends in a vowel, the ending is <TT>-yó</TT> rather than <TT>-ió</TT>:
<ul>
<TT>leer ==> leyó&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;construir ==> construyó&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;corroer ==> corroyó</TT>
</ul>
<b>However</b>, this does <b>not</b> apply to verbs ending in <TT>-guir</TT> or <TT>-quir</TT> (which are regular).
<ul>
<TT>distinguir ==> distinguió&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;delinquir ==> delinquió</TT>
</ul>
<li> <b>Vowel raising:</b> For <TT>-ir</TT> verbs only, if the stem ends in an <TT>e</TT> followed by any number of consonants, then the <TT>e</TT> changes to an <TT>i</TT>:
<ul>
<TT>pedir ==> pidió&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;sentir ==> sintió&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;presentir ==> presintió</TT>
</ul>
This <b>also</b> applies to verbs ending in <TT>-eguir</TT>:
<ul>
<TT>seguir ==> siguió&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;conseguir ==> consiguió</TT>
</ul>
</ol>

Note that Rules 1 and 3 may both apply to some verbs: <TT>heñir ==> hiñó</TT>.

## <font color='red'>TODO:</font> Build Your FST [40 points]

We have provided you with a rudimentary implementation for transducing verbs to their past tense form $-$ in particular, it only works on regular verbs. Your task is to modify the function <TT>buildFST()</TT> below to produce a better FST. You are free to define your own character sets or other helper methods that you require.

The FST will finish its analysis for a string after having read its last character. If it accepts the string, it will output the transduced form; if it does not accept it, the FST will output FAIL. Your FST may be non-deterministic (that is, a state may have multiple transitions for the same input character), but each accepted string should only have one analysis. If your FST accepts a string along multiple paths, all of the outputs will be returned and the translation will be marked incorrect: e.g. <TT>hablar ==> hablóhabló</TT>. It will accept a string if it reaches a final state after having read its last character.

<font color='green'><b>Hints:</b></font>
* <font color='green'>Before beginning, make sure you understand the "Tutorial: Designing an FST" video on Coursera. As in that video, you will probably find a non-deterministic FST easiest for this task.</font>
* <font color='green'>You should be able to complete this task with 15-20 states. You are welcome to use more, but if you find yourself with significantly more states you may be on the wrong track.</font>
* <font color='green'>As you work, keep a hand-drawn diagram of your FST on paper. You will find it easier to reason about your FST's behavior by looking at your diagram.</font>
* <font color='green'>Implement the rules in the order they're described above, and test frequently. It will help to build your FST incrementally, rather than trying to do it all at once.</font>
* <font color='green'>The exceptions to rules 2 & 3 (i.e. verbs ending in <TT>-guir</TT> & <TT>-quir</TT>) could be challenging. You might consider implementing these at the very end once all the rules themselves work (these exceptions are not worth as many points as the rules).</font>
* <font color='green'>You will sometimes want to transition on all letters except for a few. For example, suppose you want to transition from `q1` to `q2` on any letter except `e`, `g`, and `q`. You can do this by using set subtraction notation in Python: `f.addSetTransition('q1', A2Z-{'e', 'g', 'q'}, 'q2')`.</font>

In [ ]:
# Here are some predefined character sets that might come in handy. Feel free to make your own.
A2Z = set('abcçdefghijklmnopqrstuvwxyzáéíóúñü')
VOWS = set('aeiouáéíóúü')
CONS = A2Z-VOWS

# TODO: Impelement your solution here.
def buildFST():

    # The states (you need to add more)
    f = FST('q1') # q1 is the initial (non-accepting) state
    f.addState('q_n')
    f.addState('q_r')
    f.addState('q_vowel')
    f.addState('q_gq')
    f.addState('q_e')
    f.addState('q_eg')
    f.addState('q_egu')
    f.addState('q_en')
    f.addState('q_e_only')
    f.addState('q_after_e')
    f.addState('q_n_after_e')
    f.addState('q_gq_after_e')
    f.addState('q_epsilon')
    f.addState('q_EOW', True) # An accepting state

    # The transitions (you need to add more)
    f.addSetTransition('q1', A2Z-set('ñ')-VOWS-{'g','q'}, 'q1') # Self-loop on all letters to transition through stem

    f.addTransition('q1', 'ñ', 'ñ', 'q_n')
    f.addTransition('q_n', 'a', 'ó', 'q_r')
    f.addTransition('q_n', 'e', 'ó', 'q_r')
    f.addTransition('q_n', 'i', 'ó', 'q_r')
    f.addSetTransition('q_n', {'a', 'i', 'o', 'u'}, 'q_vowel')
    f.addTransition('q_n', 'e', 'e', 'q_e_only')
    #f.addSetTransition('q_n', CONS,'q1')

    f.addSetTransition('q1', VOWS-{'e'}, 'q_vowel')
    f.addSetTransition('q_vowel', VOWS-{'e'}, 'q_vowel')
    f.addTransition('q_vowel', 'e', 'e', 'q_e_only')
    f.addSetTransition('q_vowel', CONS-{'ñ', 'g', 'q'}, 'q1')
    f.addSetTransition('q_vowel',{'g','q'},'q_gq')
    f.addTransition('q_vowel', 'ñ', 'ñ', 'q_n')
    f.addSetTransition('q_n', VOWS-{'e'}, 'q_vowel')
    f.addTransition('q_n', 'e', 'e', 'q_e_only')
    f.addTransition('q_vowel', 'a', 'ó', 'q_r')
    f.addTransition('q_vowel', 'e', 'yó', 'q_r')
    f.addTransition('q_vowel', 'i', 'yó', 'q_r')
    f.addTransition('q_vowel', 'e', 'i', 'q_e')

    f.addTransition('q1', 'e', 'e', 'q_e_only')
    f.addTransition('q_e_only', 'e', 'e', 'q_e_only')
    f.addSetTransition('q_e_only', {'a', 'i', 'o', 'u'}, 'q_vowel')
    f.addSetTransition('q_e_only', CONS-{'ñ', 'g', 'q'}, 'q_after_e')
    f.addSetTransition('q_e_only',{'g','q'},'q_gq_after_e')
    f.addTransition('q_e_only', 'ñ', 'ñ', 'q_n_after_e')
    f.addTransition('q_n', 'e', 'e', 'q_e_only')
    f.addTransition('q_gq', 'e', 'e', 'q_e_only')
    f.addTransition('q_e_only', 'a', 'ó', 'q_r')
    f.addTransition('q_e_only', 'e', 'yó', 'q_r')
    f.addTransition('q_e_only', 'i', 'yó', 'q_r')
    # f.addTransition('q_e_only', 'e', 'i', 'q_e')

    f.addTransition('q_after_e', 'e', 'e', 'q_e_only')
    f.addSetTransition('q_after_e', {'a','i','o','u'}, 'q_vowel')
    f.addSetTransition('q_after_e', CONS-{'g','q','ñ'}, 'q_after_e')
    f.addTransition('q_after_e', 'a', 'ó', 'q_r')
    f.addTransition('q_after_e', 'e', 'ió', 'q_r')
    #f.addTransition('q_after_e', 'i', 'ió', 'q_r')
    f.addTransition('q_after_e', 'e', 'i', 'q_e')
    f.addSetTransition('q_after_e', {'g', 'q'}, 'q_gq')
    f.addTransition('q_after_e','ñ','ñ','q_n_after_e')

    f.addTransition('q_gq_after_e', 'a', 'ó', 'q_r')
    f.addTransition('q_gq_after_e', 'e', 'ió', 'q_r')
    # f.addTransition('q_gq', 'i', 'ió', 'q_r')
    f.addSetTransition('q_gq_after_e', {'a', 'i', 'o'}, 'q_vowel')
    f.addTransition('q_gq_after_e', 'e', 'e', 'q_e_only')
    f.addTransition('q_gq_after_e', 'e', 'i', 'q_e')
    f.addSetTransition('q_gq_after_e', CONS|set('u')|set('ü'), 'q_after_e')

    f.addTransition('q_n_after_e', 'a', 'ó', 'q_r')
    f.addTransition('q_n_after_e', 'e', 'ó', 'q_r')
    # f.addTransition('q_n', 'i', 'ó', 'q_r')
    f.addSetTransition('q_n_after_e', {'a', 'i', 'o', 'u'}, 'q_vowel')
    f.addTransition('q_n_after_e', 'e', 'e', 'q_e_only')
    # f.addSetTransition('q_n_after_e', CONS,'q1')

    f.addSetTransition('q1', {'g', 'q'}, 'q_gq')
    f.addTransition('q_gq', 'a', 'ó', 'q_r')
    f.addTransition('q_gq', 'e', 'ió', 'q_r')
    f.addTransition('q_gq', 'i', 'ió', 'q_r')
    f.addSetTransition('q_gq', {'a', 'i', 'o'}, 'q_vowel')
    f.addTransition('q_gq', 'e', 'e', 'q_e_only')
    f.addTransition('q_gq', 'e', 'i', 'q_e')
    f.addSetTransition('q_gq', CONS|set('u')|set('ü'), 'q1')

    f.addTransition('q1', 'e', 'i', 'q_e')
    f.addSetTransition('q_e', CONS-{'ñ'}, 'q_e')
    f.addTransition('q_e', 'i', 'ió', 'q_r')
    f.addTransition('q_e', 'g', 'g', 'q_eg')
    f.addTransition('q_eg', 'u', 'u', 'q_egu')
    f.addTransition('q_egu', 'i', 'ió', 'q_r')
    f.addTransition('q_e', 'ñ', 'ñ', 'q_en')
    f.addTransition('q_en', 'i', 'ó', 'q_r')

    f.addTransition('q1', 'a', 'ó', 'q_r') # Transition on vowel of ending, with a ==> ó and e ==> ió and i ==> ió
    f.addTransition('q1', 'e', 'ió', 'q_r')
    f.addTransition('q1', 'i', 'ió', 'q_r')

    f.addTransition('q_r', 'r', '', 'q_epsilon') # Transition on final r in infinitive - replace with empty string
    f.addTransition('q_epsilon', '', '', 'q_EOW') # If see empty string, at end of word

    # Return your completed FST
    return f

## Test your FST
We have provided you with a dataset to help you debug and test your FST. Run the cell below to see what verbs your FST handles correctly and your overall score. To get a high score, you must perform well on each of the rules. If you are interested, you can find the scoring function in the "Provided Code" section above.

<b>Note that the autograder uses the same scoring function on a different (hidden) dataset with similar verbs.</b> Since you don't have access to these verbs, you will want to do most of your debugging here. The autograder will provide you with a scorecard to show you what types of verbs you are getting wrong.



The `print_examples` argument of the `testFST` method may be set to one of the following values:
* `all`: Print your FST's output on all of the words in the dataset.
* `incorrect`: Print your FST's output on only the words your FST gets incorrect.
* `none`: Don't print any of the words (only show the scorecard and score).

In [ ]:
if __name__ == '__main__':
    testFST(print_examples='incorrect')

3295 / 3295 = 100.0% of examples parsed

Scorecard:
+-----------------------------------------------+-----------------------------+-----------+---------+----------------+
| Category                                      | Example                     |   Correct |   Total |   Accuracy (%) |
|-----------------------------------------------+-----------------------------+-----------+---------+----------------|
| 0a* Regular -Car verb (stem end in consonant) | hablar      ==>  habló      |      2371 |    2371 |            100 |
| 0b* Regular -Var verb (stem end in vowel)     | pasear      ==>  paseó      |       559 |     559 |            100 |
| 0c* Regular -er verb                          | comer       ==>  comió      |       150 |     150 |            100 |
| 0d* Regular -ir verb (excluding -guir, -quir) | abrir       ==>  abrió      |       130 |     130 |            100 |
| 1a  Verbs in -ñer                             | tañer       ==>  tañó       |         2 |       2 |            10

## Food for Thought
The specification above works for most Spanish verbs. Here is a brief list of some verbs that it will not work on:</b>
<ul>
<li> <b>Stems ending in <TT>ll</TT>:</b> Technically, Rule 1 above should apply to verbs whose stems end in <TT>ll</TT> as well: e.g. <TT>bullir ==> bulló</TT>
<li> <b>Verbs with <TT>o ==> u</TT> vowel raise:</b> Similar to Rule 3 above, a handful of verbs change <TT>o</TT> to <TT>u</TT> when the stem ends in <TT>o</TT> followed by any number of consonants: e.g. <TT>dormir ==> durmió</TT>
<li> <b>Rule 3 Exceptions:</b> A handful of verbs that meet the condition of Rule 3 do not raise <TT>e ==> i</TT>: e.g. <TT>discernir ==> discernió</TT>
<li> <b>Verbs in <TT>-ír</TT>:</b> These generally follow the rules of <TT>-ir</TT> verbs, except for those in <TT>-eír</TT>: e.g. <TT>reír ==> rió</TT>
<li> <b>Stem-changing verbs:</b> Verbs that are suppletive (use a different stem) in the past tense: e.g. <TT>estar ==> estuvo</TT>
<li> <b>Irregular verbs:</b> Verbs that are totally irregular: e.g. <TT>ser ==> fue</TT>
</ul>

# What to Submit

To submit the assignment, download this notebook as a <TT>.py</TT> file. You can do this by going to <TT>File > Download > Download .py</TT>. Then submit it to the autograder in Gradescope. <b>Do not try to submit it as a <TT>.ipynb</TT> file!</b>

Note that it should take <b>less than 10 minutes</b> to see your score after you have submitted to Gradescope. If your submission runs significantly longer than that, you probably have inefficiency issues in your code!